In [421]:
from transformers import pipeline
import random
import csv
from tqdm import tqdm

In [422]:
format = [["xpand 0-15", "positive"],
          ["xpand 52-68", "negative"]]

In [304]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')
filter = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ,1234567890"

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [259]:
def multiple_unmask(text, word_randomness):
    text = text.split("[MASK]")
    completed = text[0]
    
    for x in range(len(text) - 1):
        completed += "[MASK]" + text[x + 1]
        completed = unmasker(completed)[word_randomness]["sequence"]
    
    return completed

In [260]:
def clean(text):
    cleaned = ""
    
    for x in range(len(text)):
        if text[x] in filter:
            cleaned += text[x]
    
    return cleaned

In [261]:
def rephrase(text, word_randomness_0to4):
    ipt = text
    text = clean(text)
    text = text.split()
    
    while True:
        for x in range(int(len(text) / 2)):
            if random.randint(0, 2) == 0:
                text[x * 2] = "[MASK]"
        
        out = multiple_unmask(" ".join(text), word_randomness_0to4)
        
        if clean(out) != clean(ipt):
            return out

In [426]:
def expand_txt(fpath):
    file = open(fpath, "r")
    lines = file.read().split("\n")
    expanded = []
    
    for x in tqdm(range(len(lines))):
        expanded.append(rephrase(lines[x], 4))

    print("finished expanding")
    
    file = open(fpath, "a+")

    for x in tqdm(expanded):
        file.write("\n" + x)

    print("finished writing")

In [429]:
expand_txt(r"C:\Users\user\Desktop\simulation\test.txt")

100%|██████████| 100/100 [00:08<00:00, 11.37it/s]


finished expanding


100%|██████████| 100/100 [00:00<00:00, 199633.70it/s]

finished writing
